In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests
import pandas as pd

In [2]:
df = pd.read_csv('/content/training_labels2and3.csv')
df

,Unnamed: 0,id,tweets,labels
0,0,123,UPDATE: Vacaville Police K9 Punched in the Fac...,2
1,1,134,"New York police department,made in Israel!! Wa...",2
2,2,177,Bank holiday brawl between police and pub punt...,2
3,3,186,@PublicFreakout_ That’s attempted murder. Thre...,3
4,4,189,5 Bangladeshi nationals arrested in Bengaluru ...,2
...,...,...,...,...
314,314,6134,#SriLankan #Police arresting two old ladies at...,2
315,315,6135,The NYPD brutalized members of the community t...,2
316,316,6136,Police manhandle university professor during h...,2
317,317,6137,Police showed up and put this guy in a headlock,2


In [3]:
data = df['tweets']
data[0]

'UPDATE: Vacaville Police K9 Punched in the Face by Officer: Isolated Incident or Collective Canine Crisis? https://t.co/KsP2a1uvip'

In [4]:
data = " ".join(data)
data

'UPDATE: Vacaville Police K9 Punched in the Face by Officer: Isolated Incident or Collective Canine Crisis? https://t.co/KsP2a1uvip New York police department,made in Israel!! Watch as officers of the NYPD brutally attack Palestinian supporters during a peacful march. #ethniccleansing #Palestine #SaveSheikhJarrah #SaveSilwan #FreePalestine 🇵🇸 #USA #nypd #Brutality #Israel #terrorism https://t.co/5ldIyunHvH Bank holiday brawl between police and pub punters goes viral https://t.co/MLQQ9gx0k8 @PublicFreakout_ That’s attempted murder. Three Police officers. ...smashing and kneeing the person’s head. Police brutality. Fxxcking disgusting 5 Bangladeshi nationals arrested in Bengaluru BY Bengaluru police for raping, assaulting woman. #WeWantNRC Gm39 4 volunteers with Black Protest Legal Support were arrested by the Met during the #KillTheBill protests in the Spring.The arrests were challenged by Liberty,as being a breach of the legal right to independently monitor police conduct at protests.T

In [5]:
def clean_text(doc):
  tokens = doc.split()
  # removing punctuations
  table = str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens] # tokens devoid of punctuation
  #removing special characters
  tokens = [word for word in tokens if word.isalpha()]
  # convert to lowercase
  tokens = [word.lower() for word in tokens]
  return tokens

In [6]:
tokens = clean_text(data)
print(tokens[:50])

['update', 'vacaville', 'police', 'punched', 'in', 'the', 'face', 'by', 'officer', 'isolated', 'incident', 'or', 'collective', 'canine', 'crisis', 'new', 'york', 'police', 'departmentmade', 'in', 'israel', 'watch', 'as', 'officers', 'of', 'the', 'nypd', 'brutally', 'attack', 'palestinian', 'supporters', 'during', 'a', 'peacful', 'march', 'ethniccleansing', 'palestine', 'savesheikhjarrah', 'savesilwan', 'freepalestine', 'usa', 'nypd', 'brutality', 'israel', 'terrorism', 'bank', 'holiday', 'brawl', 'between', 'police']


In [7]:
len(tokens)

14554

In [8]:
len(set(tokens))

2524

In [9]:
length =50 + 1
lines = []

for i in range(length,len(tokens)):
  seq = tokens[i-length:i]
  line = " ".join(seq)
  lines.append(line)
  #if i > 3600:
  #  break

print(len(lines))

14503


In [10]:
lines[0]

'update vacaville police punched in the face by officer isolated incident or collective canine crisis new york police departmentmade in israel watch as officers of the nypd brutally attack palestinian supporters during a peacful march ethniccleansing palestine savesheikhjarrah savesilwan freepalestine usa nypd brutality israel terrorism bank holiday brawl between police and'

In [11]:
tokens[0]

'update'

In [12]:
tokens[50]

'and'

In [13]:
lines[1]

'vacaville police punched in the face by officer isolated incident or collective canine crisis new york police departmentmade in israel watch as officers of the nypd brutally attack palestinian supporters during a peacful march ethniccleansing palestine savesheikhjarrah savesilwan freepalestine usa nypd brutality israel terrorism bank holiday brawl between police and pub'

In [14]:
# prepare X and y

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
# Word embedding aka converting tokens to integers
sequences = tokenizer.texts_to_sequences(lines)


In [16]:
#convert sequences to numpy array
sequences = np.array(sequences)

In [17]:
X,y = sequences[:,:-1], sequences[:,-1]

In [18]:
X[0]

array([2524, 2523,    4,  146,    7,    1,  174,   19,   10, 2520,   89,
        145, 2519, 2518, 2517, 1187, 2516,    4, 2515,    7, 1188, 2514,
         39,   15,    6,    1,  284, 1183,  330,  772,  771,   69,    2,
       2508,  544, 2507, 2506, 2505, 2503, 2502, 2501,  284, 1181, 1188,
       2500,  770, 1180,  769,  173,    4])

In [19]:
y[0]

3

In [20]:
vocab_size = len(tokenizer.word_index) + 1

In [21]:
y = to_categorical(y, num_classes = vocab_size)
sequence_length = X.shape[1]

In [22]:
#LSTM Model

model = Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=50,input_length= sequence_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            126250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 2525)              255025    
Total params: 532,175
Trainable params: 532,175
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [24]:
model.fit(X,y,batch_size= 100, epochs = 340)

Epoch 1/340
146/146 [==============================] - 34s 212ms/step - loss: 6.5555 - accuracy: 0.0603
Epoch 2/340
146/146 [==============================] - 31s 211ms/step - loss: 6.1694 - accuracy: 0.0631
Epoch 3/340
146/146 [==============================] - 31s 211ms/step - loss: 6.0558 - accuracy: 0.0631
Epoch 4/340
146/146 [==============================] - 31s 212ms/step - loss: 5.9260 - accuracy: 0.0678
Epoch 5/340
146/146 [==============================] - 31s 214ms/step - loss: 5.7762 - accuracy: 0.0724
Epoch 6/340
146/146 [==============================] - 31s 213ms/step - loss: 5.6745 - accuracy: 0.0738
Epoch 7/340
146/146 [==============================] - 31s 210ms/step - loss: 5.5854 - accuracy: 0.0781
Epoch 8/340
146/146 [==============================] - 31s 211ms/step - loss: 5.4859 - accuracy: 0.0875
Epoch 9/340
146/146 [==============================] - 31s 211ms/step - loss: 5.3930 - accuracy: 0.0912
Epoch 10/340
146/146 [==============================] - 31s 210m

KeyboardInterrupt: ignored

In [25]:
#randomly using a line as the seed
seed_text = lines[12345]

In [26]:
def generate_text_seq(model,tokenizer,text_seq_length,seed_text,n_words):
  text =[]

  for _ in range(n_words):
    # text_to_sequences: provides a tuple of {words:integer}
    # we need only the integer number of seed_text and thus the 0th element
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    # encoded may have more than 50 words so we need to truncate it at 50th word and thus pad the sequence
    encoded = pad_sequences([encoded],maxlen = text_seq_length, truncating ='pre')

    #predict class for each word which is an integer value
    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    # now we match the y_predict integer with the integer from the tokenizer
      # if we find a match we grab the related word as the predicted_word
    for word ,index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break

    # this predicted word is added to the seed text
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return " ".join(text) 

In [27]:
generate_text_seq(model,tokenizer,sequence_length,seed_text,30)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'block of sw alder st one officer strikes a protester repeatedly with their baton that protester was later arrested at an antiice protest in the portland pearl district several police'